In [124]:
import torch
import torch.nn as nn
import numpy as np

# Parameters
nu = 0.01  # Dynamic viscosity
rho = 1.0  # Fluid density
beta = 1.0 # Constant for the divergence term
V = 1.0 # Inlet velocity

# Define a class to solve steady-state incompressible Navier-Stokes equations using PINN
class NavierStokes3D():
    def __init__(self, X, Y, Z, u, v, w, p):
        self.x = torch.tensor(X, dtype=torch.float32, requires_grad=True)
        self.y = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
        self.z = torch.tensor(Z, dtype=torch.float32, requires_grad=True)
        self.u = torch.tensor(u, dtype=torch.float32)
        self.v = torch.tensor(v, dtype=torch.float32)
        self.w = torch.tensor(w, dtype=torch.float32)
        self.p = torch.tensor(p, dtype=torch.float32)
        self.null = torch.zeros((self.x.shape[0], 1))
        self.network()

        # Optimizer
        self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr=1, max_iter=10000)
        self.mse = nn.MSELoss()
        self.ls = 0
        self.iter = 0

    def network(self):
        self.net = nn.Sequential(
            nn.Linear(3, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 4))  # Output: u, v, w, p

    def function(self, x, y, z):
        # Get the predictions for velocity and pressure
        res = self.net(torch.hstack((x, y, z)))
        u, v, w, p = res[:, 0:1], res[:, 1:2], res[:, 2:3], res[:, 3:4]

        # Compute the spatial derivatives
        u_x, u_xx, u_y, u_yy, u_z, u_zz = [torch.autograd.grad(u, var, grad_outputs=torch.ones_like(u), create_graph=True)[0] for var in [x, x, y, y, z, z]]
        v_x, v_xx, v_y, v_yy, v_z, v_zz = [torch.autograd.grad(v, var, grad_outputs=torch.ones_like(v), create_graph=True)[0] for var in [x, x, y, y, z, z]]
        w_x, w_xx, w_y, w_yy, w_z, w_zz = [torch.autograd.grad(w, var, grad_outputs=torch.ones_like(w), create_graph=True)[0] for var in [x, x, y, y, z, z]]
        p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        p_z = torch.autograd.grad(p, z, grad_outputs=torch.ones_like(p), create_graph=True)[0]

        # PDE Residuals (Steady-state Incompressible NS Equations)
        f_u = rho * (u * u_x + v * u_y + w * u_z) + p_x - nu * (u_xx + u_yy + u_zz)
        f_v = rho * (u * v_x + v * v_y + w * v_z) + p_y - nu * (v_xx + v_yy + v_zz)
        f_w = rho * (u * w_x + v * w_y + w * w_z) + p_z - nu * (w_xx + w_yy + w_zz)
        div_u = u_x + v_y + w_z

        return u, v, w, p, f_u, f_v, f_w, div_u

    def boundary_condition_loss(self, x, y, z, boundary_type):
        # Boundary conditions for walls, inlet, and outlet
        if boundary_type == "wall":
            u_loss = self.mse(self.u, torch.zeros_like(self.u))
            v_loss = self.mse(self.v, torch.zeros_like(self.v))
            w_loss = self.mse(self.w, torch.zeros_like(self.w))
            p_loss = self.mse(self.p, torch.zeros_like(self.p))
        elif boundary_type == "inlet":
            u_loss = self.mse(self.u, torch.tensor(V, dtype=torch.float32))
            v_loss = self.mse(self.v, torch.zeros_like(self.v))
            w_loss = self.mse(self.w, torch.zeros_like(self.w))
            p_loss = self.mse(self.p, torch.zeros_like(self.p))
        elif boundary_type == "outlet":
            # Compute the boundary flux term: (-pI + nu*grad(u))·n = 0 for outlet
            u_loss = self.mse(self.u, torch.zeros_like(self.u))
            v_loss = self.mse(self.v, torch.zeros_like(self.v))
            w_loss = self.mse(self.w, torch.zeros_like(self.w))
            p_loss = self.mse(self.p, torch.zeros_like(self.p))
        return u_loss + v_loss + w_loss + p_loss

    def closure(self):
        self.optimizer.zero_grad()
        u_prediction, v_prediction, w_prediction, p_prediction, f_u, f_v, f_w, div_u = self.function(self.x, self.y, self.z)
        u_loss = self.mse(u_prediction, self.u)
        v_loss = self.mse(v_prediction, self.v)
        w_loss = self.mse(w_prediction, self.w)
        p_loss = self.mse(p_prediction, self.p)
        f_loss = self.mse(f_u, self.null) + self.mse(f_v, self.null) + self.mse(f_w, self.null)
        div_loss = self.mse(div_u, self.null)
        
        self.ls = u_loss + v_loss + w_loss + p_loss + f_loss + div_loss
        self.ls.backward()
        self.iter += 1
        if not self.iter % 100:
            print(f'Iteration: {self.iter}, Loss: {self.ls.item():0.6f}')
        return self.ls

    def train(self):
        self.net.train()
        self.optimizer.step(self.closure)

# Example of how to initialize and train:
# Load your data
# X, Y, Z are your spatial coordinates, u, v, w are velocities, p is pressure
# Example for initializing and training would go here


In [125]:
def read_multi_solid_stl(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]  # 去除空行

    solids = {}
    current_solid = None
    current_facets = []

    i = 0
    while i < len(lines):
        line = lines[i]
        if line.startswith('solid'):
            # 开始新 solid
            solid_name = line.split(' ', 1)[1] if ' ' in line else 'unknown'
            current_solid = solid_name
            current_facets = []
        elif line.startswith('endsolid'):
            # 结束当前 solid
            if current_solid is not None:
                solids[current_solid] = current_facets
            current_solid = None
        elif current_solid is not None and line.startswith('facet'):
            # 开始读取一个三角面
            facet_lines = [line]
            i += 1
            # 读取到 endfacet 为止
            while i < len(lines) and not lines[i].startswith('endfacet'):
                facet_lines.append(lines[i])
                i += 1
            if i < len(lines):
                facet_lines.append(lines[i])  # 加上 'endfacet'
            current_facets.append(facet_lines)
            # 注意：这里不再 i += 1，因为外层循环会加
        i += 1  # 外层循环递增

    return solids

# 使用示例
stl_path = 'stl/1_1_1_1_1_1.stl'
solids = read_multi_solid_stl(stl_path)

print("检测到的 solid 部分:", list(solids.keys()))
for name, facets in solids.items():
    print(f"{name}: {len(facets)} 个三角形")

检测到的 solid 部分: ['wall', 'outlet', 'inlet']
wall: 1600 个三角形
outlet: 18 个三角形
inlet: 18 个三角形


In [126]:
from stl import mesh

def facets_to_mesh(facets):
    """将解析出的 facets 列表转换为 mesh.Mesh 对象"""
    data = np.zeros(len(facets), dtype=mesh.Mesh.dtype)
    for i, facet_lines in enumerate(facets):
        # 提取顶点行（以 vertex 开头的行）
        vertices_lines = [line for line in facet_lines if line.strip().startswith('vertex')]
        for j, vline in enumerate(vertices_lines):
            # 解析 vertex x y z
            _, x, y, z = vline.strip().split()
            data['vectors'][i][j] = [float(x), float(y), float(z)]
    return mesh.Mesh(data)

# 分别处理每个部分
meshes = {}
for name, facets in solids.items():
    meshes[name] = facets_to_mesh(facets)
    print(f"已创建 '{name}' 的 Mesh 对象，包含 {len(facets)} 个三角形")

# 示例：访问 wall 的顶点
wall_vertices = meshes['wall'].vectors
inlet_vertices = meshes['inlet'].vectors
outlet_vertices = meshes['outlet'].vectors

已创建 'wall' 的 Mesh 对象，包含 1600 个三角形
已创建 'outlet' 的 Mesh 对象，包含 18 个三角形
已创建 'inlet' 的 Mesh 对象，包含 18 个三角形


In [127]:
np.shape(wall_vertices)


(1600, 3, 3)

In [128]:
import numpy as np
from stl import mesh
import matplotlib.pyplot as plt

# 函数：在三角形内生成一个随机点
def random_point_in_triangle(triangle):
    # 计算重心坐标
    r1 = np.random.random()
    r2 = np.random.random()
    if r1 + r2 > 1:
        r1 = 1 - r1
        r2 = 1 - r2
    # 通过重心坐标计算点的位置
    point = (1 - r1 - r2) * triangle[0] + r1 * triangle[1] + r2 * triangle[2]
    return point

def surface_sampling(vertices, num):
    surface_points = []
    for triangle in vertices:
        for _ in range(num // len(vertices)):
            random_point = random_point_in_triangle(triangle)
            surface_points.append(random_point)
    return surface_points

# 判断点是否在管道内的射线法（Ray Casting）
def ray_intersect_triangle(ray_origin, ray_direction, triangle):
    v0, v1, v2 = triangle
    epsilon = 1e-6

    # 计算三角形的边向量
    e1 = v1 - v0
    e2 = v2 - v0

    # 计算法向量
    h = np.cross(ray_direction, e2)
    a = np.dot(e1, h)

    if abs(a) < epsilon:  # 如果平行，不相交
        return False

    f = 1.0 / a
    s = ray_origin - v0
    u = f * np.dot(s, h)

    if u < 0.0 or u > 1.0:  # 点在三角形外部
        return False

    q = np.cross(s, e1)
    v = f * np.dot(ray_direction, q)

    if v < 0.0 or u + v > 1.0:  # 点在三角形外部
        return False

    t = f * np.dot(e2, q)
    return t > epsilon  # 射线与三角形相交

# 使用射线法检查点是否在管道内部
def is_point_in_pipe_ray_casting(point, surface_triangles, mid):
    ray_origin = point
    if point[1] > mid:
        ray_direction = np.array([0, -1, 0])  # 假设 X 轴方向为射线方向
    else:
        ray_direction = np.array([0, 1, 0])

    intersection_count = 0
    for triangle in surface_triangles:
        if ray_intersect_triangle(ray_origin, ray_direction, triangle):
            intersection_count += 1
        if intersection_count == 2:
            return False
    
    # 如果交点数为奇数，点在管道内部；偶数则在外部
    return intersection_count % 2 == 1

# 在管道内部生成点（在整个管道的体积内）
def volume_sampling(min_coords, max_coords, num_points, surface_triangles, mid):
    points = []
    while len(points) < num_points:
        random_point = np.random.uniform(min_coords, max_coords)
        if is_point_in_pipe_ray_casting(random_point, surface_triangles, mid):
            points.append(random_point)
    
    return np.array(points)

# 读取STL文件并提取表面部分
wall_vertices = meshes['wall'].vectors
inlet_vertices = meshes['inlet'].vectors
outlet_vertices = meshes['outlet'].vectors

# 表面采样
surface_points_wall = surface_sampling(wall_vertices, 1600)   # 8000为示例数量
surface_points_inlet = surface_sampling(inlet_vertices, 38)
surface_points_outlet = surface_sampling(outlet_vertices, 38)


min_coords = np.min(surface_points_wall, axis=0)
max_coords = np.max(surface_points_wall, axis=0)

mid = (min_coords + max_coords)[1] / 2  # Y轴中点


surface_triangles = np.vstack((wall_vertices, inlet_vertices, outlet_vertices))


volume_points = volume_sampling(min_coords, max_coords, 500, wall_vertices, mid)  # 500为示例数量








In [165]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surface_points_inlet = np.array(surface_points_inlet)
surface_points_outlet = np.array(surface_points_outlet)
surface_points_wall = np.array(surface_points_wall)

# 绘制 wall 点，设置颜色为红色
ax.scatter(surface_points_wall[:, 0], surface_points_wall[:, 1], surface_points_wall[:, 2], c='r', s=1, label='Wall')

# 绘制 inlet 点，设置颜色为蓝色
ax.scatter(surface_points_inlet[:, 0], surface_points_inlet[:, 1], surface_points_inlet[:, 2], c='b', s=1, label='Inlet')

# 绘制 outlet 点，设置颜色为绿色
ax.scatter(surface_points_outlet[:, 0], surface_points_outlet[:, 1], surface_points_outlet[:, 2], c='g', s=1, label='Outlet')

# 绘制 volume 内部点，设置颜色为黄色
ax.scatter(volume_points[:, 0], volume_points[:, 1], volume_points[:, 2], c='y', s=1, label='Volume')

# 设置坐标轴标签
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')


# --- 关键步骤：设置等比例坐标轴 ---
# 获取数据范围
all_points = np.vstack((surface_points_wall, surface_points_inlet, surface_points_outlet, volume_points))
max_range = np.array([all_points[:, 0].max() - all_points[:, 0].min(),
                      all_points[:, 1].max() - all_points[:, 1].min(),
                      all_points[:, 2].max() - all_points[:, 2].min()]).max() / 2.0
# 计算中心点
mid_x = (all_points[:, 0].max() + all_points[:, 0].min()) * 0.5
mid_y = (all_points[:, 1].max() + all_points[:, 1].min()) * 0.5
mid_z = (all_points[:, 2].max() + all_points[:, 2].min()) * 0.5

# 设置每个轴的相同范围
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

# 添加图例
ax.legend()

plt.show()

In [130]:
is_point_in_pipe_ray_casting((0,0,0.2),wall_vertices,mid)

False

[1]

In [193]:
import torch
import torch.nn as nn
import numpy as np

class NavierStokes3D_PINN:
    def __init__(self, X_train):
        self.x = torch.tensor(X_train[:, 0:1], dtype=torch.float32, requires_grad=True)
        self.y = torch.tensor(X_train[:, 1:2], dtype=torch.float32, requires_grad=True)
        self.z = torch.tensor(X_train[:, 2:3], dtype=torch.float32, requires_grad=True)
        
        self.net = self.network()
        self.mse_loss = nn.MSELoss()

        # 分别定义两个优化器
        self.optimizer_adam = torch.optim.Adam(self.net.parameters(), lr=1e-3)
        # 构造函数中
        self.optimizer_lbfgs = torch.optim.LBFGS(
            self.net.parameters(),
            lr=1.0,
            max_iter=10,           # 每次 step() 只迭代一次
            max_eval=10,
            tolerance_grad=1e-10,
            tolerance_change=1e-10,
            history_size=100,
            line_search_fn='strong_wolfe'  # 可提高稳定性
        )
        
        # ✅ 添加：用于记录损失
        self.loss_log = {
            'adam': {
                'total': [],
                'pde': [],
                'wall': [],
                'inlet': [],
                'outlet': []
            },
            'lbfgs': {
                'total': [],
                'pde': [],
                'wall': [],
                'inlet': [],
                'outlet': []
            }
        }


    def network(self):
        layers = [
            nn.Linear(3, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 4)  # u, v, w, p
        ]
        return nn.Sequential(*layers)

    def forward(self, x, y, z):
        input_tensor = torch.cat([x, y, z], dim=1)
        output = self.net(input_tensor)
        u, v, w, p = output[:, 0:1], output[:, 1:2], output[:, 2:3], output[:, 3:4]
        return u, v, w, p

    def compute_pde_residuals(self, x, y, z):
        u, v, w, p = self.forward(x, y, z)
        
        # 所有导数计算保持不变...
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_z = torch.autograd.grad(u, z, grad_outputs=torch.ones_like(u), create_graph=True)[0]

        v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_z = torch.autograd.grad(v, z, grad_outputs=torch.ones_like(v), create_graph=True)[0]

        w_x = torch.autograd.grad(w, x, grad_outputs=torch.ones_like(w), create_graph=True)[0]
        w_y = torch.autograd.grad(w, y, grad_outputs=torch.ones_like(w), create_graph=True)[0]
        w_z = torch.autograd.grad(w, z, grad_outputs=torch.ones_like(w), create_graph=True)[0]

        p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        p_z = torch.autograd.grad(p, z, grad_outputs=torch.ones_like(p), create_graph=True)[0]

        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
        u_zz = torch.autograd.grad(u_z, z, grad_outputs=torch.ones_like(u_z), create_graph=True)[0]

        v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
        v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
        v_zz = torch.autograd.grad(v_z, z, grad_outputs=torch.ones_like(v_z), create_graph=True)[0]

        w_xx = torch.autograd.grad(w_x, x, grad_outputs=torch.ones_like(w_x), create_graph=True)[0]
        w_yy = torch.autograd.grad(w_y, y, grad_outputs=torch.ones_like(w_y), create_graph=True)[0]
        w_zz = torch.autograd.grad(w_z, z, grad_outputs=torch.ones_like(w_z), create_graph=True)[0]

        div_res = u_x + v_y + w_z
        f_u = rho * (u*u_x + v*u_y + w*u_z) + p_x - nu*(u_xx + u_yy + u_zz)
        f_v = rho * (u*v_x + v*v_y + w*v_z) + p_y - nu*(v_xx + v_yy + v_zz)
        f_w = rho * (u*w_x + v*w_y + w*w_z) + p_z - nu*(w_xx + w_yy + w_zz)

        return f_u, f_v, f_w, div_res

    def compute_boundary_loss(self, x_bc, y_bc, z_bc, bc_type="wall", target_vel=None):
        """统一的边界损失计算"""
        u_pred, v_pred, w_pred, p_pred = self.forward(x_bc, y_bc, z_bc)
        
        if bc_type == "wall":
            loss = (self.mse_loss(u_pred, torch.zeros_like(u_pred)) +
                    self.mse_loss(v_pred, torch.zeros_like(v_pred)) +
                    self.mse_loss(w_pred, torch.zeros_like(w_pred)))
        elif bc_type == "inlet":
            V = target_vel if target_vel is not None else V_inlet
            loss = (self.mse_loss(u_pred, torch.zeros_like(u_pred)) +
                    self.mse_loss(v_pred, torch.zeros_like(v_pred)) +
                    self.mse_loss(w_pred, -V * torch.ones_like(w_pred)))
        elif bc_type == "outlet":
            # 正确做法：出口压力 = 0
            loss = self.mse_loss(p_pred, torch.zeros_like(p_pred))
        else:
            loss = torch.tensor(0.0)
        return loss
                

    def train_adam(self, n_epochs=300):
    #"""阶段1: 使用 Adam 预热（正确打印损失）"""
        print("Starting Adam pre-training...")
    
        for epoch in range(n_epochs):
        # 清除梯度
            self.optimizer_adam.zero_grad()
        
            # 计算 PDE 残差损失
            f_u, f_v, f_w, div_res = self.compute_pde_residuals(self.x, self.y, self.z)
            loss_pde = (self.mse_loss(f_u, torch.zeros_like(f_u)) +
                        self.mse_loss(f_v, torch.zeros_like(f_v)) +
                        self.mse_loss(f_w, torch.zeros_like(f_w)) +
                        self.mse_loss(div_res, torch.zeros_like(div_res)))

            # 计算边界损失
            x_wall = torch.tensor(surface_points_wall[:, 0:1], dtype=torch.float32, requires_grad=True)
            y_wall = torch.tensor(surface_points_wall[:, 1:2], dtype=torch.float32, requires_grad=True)
            z_wall = torch.tensor(surface_points_wall[:, 2:3], dtype=torch.float32, requires_grad=True)
            loss_wall = self.compute_boundary_loss(x_wall, y_wall, z_wall, "wall")

            x_inlet = torch.tensor(surface_points_inlet[:, 0:1], dtype=torch.float32, requires_grad=True)
            y_inlet = torch.tensor(surface_points_inlet[:, 1:2], dtype=torch.float32, requires_grad=True)
            z_inlet = torch.tensor(surface_points_inlet[:, 2:3], dtype=torch.float32, requires_grad=True)
            loss_inlet = self.compute_boundary_loss(x_inlet, y_inlet, z_inlet, "inlet")

            x_outlet = torch.tensor(surface_points_outlet[:, 0:1], dtype=torch.float32, requires_grad=True)
            y_outlet = torch.tensor(surface_points_outlet[:, 1:2], dtype=torch.float32, requires_grad=True)
            z_outlet = torch.tensor(surface_points_outlet[:, 2:3], dtype=torch.float32, requires_grad=True)
            loss_outlet = self.compute_boundary_loss(x_outlet, y_outlet, z_outlet, "outlet")

            total_loss = 5 * loss_pde + 5 * loss_wall + loss_inlet + loss_outlet

            # 反向传播
            total_loss.backward()

            # 更新参数
            self.optimizer_adam.step()

            # ✅ 记录 LBFGS 损失
            if epoch % 5 == 0 or epoch == n_epochs - 1:
                self.loss_log['adam']['total'].append(total_loss.item())
                self.loss_log['adam']['pde'].append(loss_pde.item())
                self.loss_log['adam']['wall'].append(loss_wall.item())
                self.loss_log['adam']['inlet'].append(loss_inlet.item())
                self.loss_log['adam']['outlet'].append(loss_outlet.item())

            # 打印损失
            if (epoch + 1) % 10 == 0:
                print(f"Adam Epoch {epoch+1:5d}/{n_epochs}, Loss: {total_loss.item():.6e}, "
                    f"PDE: {loss_pde.item():.6e}, Wall: {loss_wall.item():.6e}, "
                    f"Inlet: {loss_inlet.item():.6e}, Outlet: {loss_outlet.item():.6e}")

    def train_lbfgs(self, n_iterations=50):
        """阶段2: 使用 LBFGS 分步精细调优（可监控进度）"""
        print("Starting LBFGS fine-tuning...")

        for it in range(n_iterations):
            def closure():
                self.optimizer_lbfgs.zero_grad()
            
                # === 计算 PDE 残差 ===
                f_u, f_v, f_w, div_res = self.compute_pde_residuals(self.x, self.y, self.z)
                loss_pde = (self.mse_loss(f_u, torch.zeros_like(f_u)) +
                        self.mse_loss(f_v, torch.zeros_like(f_v)) +
                        self.mse_loss(f_w, torch.zeros_like(f_w)) +
                        self.mse_loss(div_res, torch.zeros_like(div_res)))

                # === 边界损失 ===
                x_wall = torch.tensor(surface_points_wall[:, 0:1], dtype=torch.float32, requires_grad=True)
                y_wall = torch.tensor(surface_points_wall[:, 1:2], dtype=torch.float32, requires_grad=True)
                z_wall = torch.tensor(surface_points_wall[:, 2:3], dtype=torch.float32, requires_grad=True)
                loss_wall = self.compute_boundary_loss(x_wall, y_wall, z_wall, "wall")

                x_inlet = torch.tensor(surface_points_inlet[:, 0:1], dtype=torch.float32, requires_grad=True)
                y_inlet = torch.tensor(surface_points_inlet[:, 1:2], dtype=torch.float32, requires_grad=True)
                z_inlet = torch.tensor(surface_points_inlet[:, 2:3], dtype=torch.float32, requires_grad=True)
                loss_inlet = self.compute_boundary_loss(x_inlet, y_inlet, z_inlet, "inlet")

                x_outlet = torch.tensor(surface_points_outlet[:, 0:1], dtype=torch.float32, requires_grad=True)
                y_outlet = torch.tensor(surface_points_outlet[:, 1:2], dtype=torch.float32, requires_grad=True)
                z_outlet = torch.tensor(surface_points_outlet[:, 2:3], dtype=torch.float32, requires_grad=True)
                loss_outlet = self.compute_boundary_loss(x_outlet, y_outlet, z_outlet, "outlet")

                total_loss = 5 * loss_pde + 5 * loss_wall + loss_inlet + loss_outlet
                total_loss.backward()

                # ✅ 记录 LBFGS 损失
                if it % 5 == 0 or it == n_iterations - 1:
                    self.loss_log['lbfgs']['total'].append(total_loss.item())
                    self.loss_log['lbfgs']['pde'].append(loss_pde.item())
                    self.loss_log['lbfgs']['wall'].append(loss_wall.item())
                    self.loss_log['lbfgs']['inlet'].append(loss_inlet.item())
                    self.loss_log['lbfgs']['outlet'].append(loss_outlet.item())

                # 打印当前损失
                if it % 5 == 0 or it == n_iterations - 1:
                    print(f"LBFGS Iter {it+1:3d}/{n_iterations} | Total: {total_loss.item():.6e} | "
                        f"PDE: {loss_pde.item():.6e} | Wall: {loss_wall.item():.6e} | "
                        f"Inlet:{loss_inlet.item():.6e} | Outlet:{loss_outlet.item():.6e}")
                return total_loss

            # 关键：每次只让 LBFGS 做一次“尝试”
            self.optimizer_lbfgs.step(closure)

    def train(self):
        """一键启动两阶段训练"""
        self.train_adam(n_epochs=400)      # 每200步打印一次
        self.train_lbfgs(n_iterations=80)    # LBFGS 迭代5次，每次打印

    def plot_convergence(self):
        plt.figure(figsize=(12, 6))

        # 获取记录的损失
        adam_total = self.loss_log['adam']['total']
        adam_pde = self.loss_log['adam']['pde']
        adam_wall = self.loss_log['adam']['wall']
        adam_inlet = self.loss_log['adam']['inlet']
        adam_outlet = self.loss_log['adam']['outlet']

        lbfgs_total = self.loss_log['lbfgs']['total']
        lbfgs_pde = self.loss_log['lbfgs']['pde']
        lbfgs_wall = self.loss_log['lbfgs']['wall']
        lbfgs_inlet = self.loss_log['lbfgs']['inlet']
        lbfgs_outlet = self.loss_log['lbfgs']['outlet']

        # 创建 x 轴：Adam 步数 + LBFGS 步数
        adam_epochs = np.arange(0, len(adam_total)) * 5  # 每100步记录一次
        lbfgs_steps = np.arange(0, len(lbfgs_total)) * 5 + max(adam_epochs)  # 假设每10步记录

        # 画图
        plt.semilogy(adam_epochs, adam_total, 'k-', label='Total (Adam)', linewidth=2)
        plt.semilogy(adam_epochs, adam_pde, 'r--', label='PDE (Adam)')
        plt.semilogy(adam_epochs, adam_wall, 'b--', label='Wall (Adam)')
        plt.semilogy(adam_epochs, adam_inlet, 'g--', label='Inlet (Adam)')
        plt.semilogy(adam_epochs, adam_outlet, 'm--', label='Outlet (Adam)')

        plt.semilogy(lbfgs_steps, lbfgs_total, 'k-', label='Total (Final)',linewidth=2)  # 总损失延续
        plt.semilogy(lbfgs_steps, lbfgs_pde, 'r:', label='PDE (LBFGS)')
        plt.semilogy(lbfgs_steps, lbfgs_wall, 'b:', label='Wall (LBFGS)')
        plt.semilogy(lbfgs_steps, lbfgs_inlet, 'g:', label='Inlet (LBFGS)')
        plt.semilogy(lbfgs_steps, lbfgs_outlet, 'm:', label='Outlet (Outlet)')

        plt.xlabel('Training Step')
        plt.ylabel('Loss (log scale)')
        plt.title('PINN Training Convergence: Adam + LBFGS')
        plt.legend()
        plt.grid(True, which="both", ls="--", alpha=0.5)
        plt.tight_layout()
        plt.show()

        # ========= 新增：在任意体内点上做预测 =========
    def predict_on_points(self, volume_points):
        """
        volume_points: (N, 3) 的 numpy 数组或 torch.Tensor，每行为 [x, y, z]
        返回: x,y,z,u,v,w,p，都是 numpy 一维数组 (N,)
        """
        # 如果是 torch.Tensor 就先转成 numpy
        if isinstance(volume_points, torch.Tensor):
            volume_points = volume_points.detach().cpu().numpy()

        device = next(self.net.parameters()).device  # 支持以后把 net.to('cuda')

        x = torch.tensor(volume_points[:, 0:1], dtype=torch.float32, device=device)
        y = torch.tensor(volume_points[:, 1:2], dtype=torch.float32, device=device)
        z = torch.tensor(volume_points[:, 2:3], dtype=torch.float32, device=device)

        # 推理阶段不需要梯度
        with torch.no_grad():
            u, v, w, p = self.forward(x, y, z)

        # 全部转成 numpy 一维数组，方便 matplotlib 使用
        x_np = x.detach().cpu().numpy().flatten()
        y_np = y.detach().cpu().numpy().flatten()
        z_np = z.detach().cpu().numpy().flatten()
        u_np = u.detach().cpu().numpy().flatten()
        v_np = v.detach().cpu().numpy().flatten()
        w_np = w.detach().cpu().numpy().flatten()
        p_np = p.detach().cpu().numpy().flatten()

        return x_np, y_np, z_np, u_np, v_np, w_np, p_np









In [194]:
# 假设你已经用某种方式采样了这些点
# volume_points: (N_vol, 3)  体积域内的随机采样点
# surface_points_wall: (N_wall, 3)  壁面边界点
# surface_points_inlet: (N_inlet, 3) 入口边界点
# surface_points_outlet: (N_outlet, 3) 出口边界点

# 合并所有训练点（PDE 残差会在所有点上计算，但也可以只在 volume_points 上计算）
X_train = np.vstack([volume_points, 
                     surface_points_wall, 
                     surface_points_inlet, 
                     surface_points_outlet])



# 初始化模型
pinn_model = NavierStokes3D_PINN(X_train)

# 开始训练
pinn_model.train()

Starting Adam pre-training...
Adam Epoch    10/400, Loss: 8.063715e-01, PDE: 1.315218e-04, Wall: 3.787427e-02, Inlet: 6.162084e-01, Outlet: 1.341206e-04
Adam Epoch    20/400, Loss: 6.010745e-01, PDE: 7.532433e-03, Wall: 4.141431e-02, Inlet: 3.563228e-01, Outlet: 1.800203e-05
Adam Epoch    30/400, Loss: 4.804453e-01, PDE: 7.782585e-03, Wall: 2.614765e-02, Inlet: 3.107700e-01, Outlet: 2.403886e-05
Adam Epoch    40/400, Loss: 4.053865e-01, PDE: 1.213267e-02, Wall: 3.037590e-02, Inlet: 1.927434e-01, Outlet: 1.003155e-04
Adam Epoch    50/400, Loss: 3.464698e-01, PDE: 1.013579e-02, Wall: 2.706090e-02, Inlet: 1.602485e-01, Outlet: 2.378694e-04
Adam Epoch    60/400, Loss: 3.099815e-01, PDE: 9.629894e-03, Wall: 2.123621e-02, Inlet: 1.555971e-01, Outlet: 5.385056e-05
Adam Epoch    70/400, Loss: 2.900619e-01, PDE: 1.315196e-02, Wall: 2.782390e-02, Inlet: 8.506402e-02, Outlet: 1.186057e-04
Adam Epoch    80/400, Loss: 2.687288e-01, PDE: 1.205818e-02, Wall: 2.204103e-02, Inlet: 9.816631e-02, Outlet:

In [195]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 用已经训练好的 pinn 做预测
def predict_on_points(model, volume_points):
    if isinstance(volume_points, torch.Tensor):
        volume_points = volume_points.detach().cpu().numpy()

    device = next(model.net.parameters()).device

    x = torch.tensor(volume_points[:, 0:1], dtype=torch.float32, device=device)
    y = torch.tensor(volume_points[:, 1:2], dtype=torch.float32, device=device)
    z = torch.tensor(volume_points[:, 2:3], dtype=torch.float32, device=device)
    #surface_points_inlet
    with torch.no_grad():
        u, v, w, p = model.forward(x, y, z)

    x_np = x.detach().cpu().numpy().flatten()
    y_np = y.detach().cpu().numpy().flatten()
    z_np = z.detach().cpu().numpy().flatten()
    u_np = u.detach().cpu().numpy().flatten()
    v_np = v.detach().cpu().numpy().flatten()
    w_np = w.detach().cpu().numpy().flatten()
    p_np = p.detach().cpu().numpy().flatten()

    return x_np, y_np, z_np, u_np, v_np, w_np, p_np


def plot_velocity_vectors(model, volume_points, step=2, arrow_length=0.5, normalize=True):
    x, y, z, u, v, w, p = predict_on_points(model, volume_points)

    idx = np.arange(0, x.shape[0], step)
    x_s, y_s, z_s = x[idx], y[idx], z[idx]
    u_s, v_s, w_s = u[idx], v[idx], w[idx]

    if normalize:
        mag = np.sqrt(u_s**2 + v_s**2 + w_s**2) + 1e-8
        u_s = u_s / mag
        v_s = v_s / mag
        w_s = w_s / mag

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(x_s, y_s, z_s, u_s, v_s, w_s, length=arrow_length, linewidth=0.5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Velocity Vectors')

    # 等比例坐标
    x_min, x_max = x_s.min(), x_s.max()
    y_min, y_max = y_s.min(), y_s.max()
    z_min, z_max = z_s.min(), z_s.max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    z_range = z_max - z_min
    max_range = max(x_range, y_range, z_range)
    x_mid = 0.5 * (x_max + x_min)
    y_mid = 0.5 * (y_max + y_min)
    z_mid = 0.5 * (z_max + z_min)
    ax.set_xlim(x_mid - max_range / 2, x_mid + max_range / 2)
    ax.set_ylim(y_mid - max_range / 2, y_mid + max_range / 2)
    ax.set_zlim(z_mid - max_range / 2, z_mid + max_range / 2)

    plt.tight_layout()
    plt.show()


def plot_pressure_cloud(model, volume_points, s=5):
    x, y, z, u, v, w, p = predict_on_points(model, volume_points)

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    sc = ax.scatter(x, y, z, c=p, s=s, cmap='jet')
    cb = plt.colorbar(sc, ax=ax, shrink=0.6)
    cb.set_label('Pressure')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Pressure Cloud')

    # 等比例坐标
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    z_min, z_max = z.min(), z.max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    z_range = z_max - z_min
    max_range = max(x_range, y_range, z_range)
    x_mid = 0.5 * (x_max + x_min)
    y_mid = 0.5 * (y_max + y_min)
    z_mid = 0.5 * (z_max + z_min)
    ax.set_xlim(x_mid - max_range / 2, x_mid + max_range / 2)
    ax.set_ylim(y_mid - max_range / 2, y_mid + max_range / 2)
    ax.set_zlim(z_mid - max_range / 2, z_mid + max_range / 2)

    plt.tight_layout()
    plt.show()

# pinn 是你已经训练好的模型实例
plot_velocity_vectors(pinn_model, volume_points, step=2, arrow_length=0.5)
plot_pressure_cloud(pinn_model, volume_points, s=5)



In [188]:
def compute_div_on_volume(model, volume_points):
    device = next(model.net.parameters()).device

    if isinstance(volume_points, torch.Tensor):
        pts = volume_points.to(device).clone().detach().requires_grad_(True)
    else:
        pts = torch.tensor(volume_points, dtype=torch.float32, device=device, requires_grad=True)

    x = pts[:, 0:1]
    y = pts[:, 1:2]
    z = pts[:, 2:3]

    u, v, w, p = model.forward(x, y, z)

    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, torch.ones_like(v), create_graph=True)[0]
    w_z = torch.autograd.grad(w, z, torch.ones_like(w), create_graph=True)[0]

    div_res = u_x + v_y + w_z

    with torch.no_grad():
        div_np = div_res.detach().cpu().numpy().flatten()
        x_np = x.detach().cpu().numpy().flatten()
        y_np = y.detach().cpu().numpy().flatten()
        z_np = z.detach().cpu().numpy().flatten()

    return x_np, y_np, z_np, div_np


In [196]:
def check_bc_errors(model):
    device = next(model.net.parameters()).device

    # ---------- wall: no-slip ----------
    x_wall = torch.tensor(surface_points_wall[:, 0:1], dtype=torch.float32, device=device)
    y_wall = torch.tensor(surface_points_wall[:, 1:2], dtype=torch.float32, device=device)
    z_wall = torch.tensor(surface_points_wall[:, 2:3], dtype=torch.float32, device=device)

    with torch.no_grad():
        u_w, v_w, w_w, p_w = model.forward(x_wall, y_wall, z_wall)

    wall_u_err = u_w.abs().mean().item()
    wall_v_err = v_w.abs().mean().item()
    wall_w_err = w_w.abs().mean().item()
    print(f"[BC-Wall] mean |u|={wall_u_err:.3e}, |v|={wall_v_err:.3e}, |w|={wall_w_err:.3e}")

    # ---------- inlet: 指定来流速度 ----------
    x_in = torch.tensor(surface_points_inlet[:, 0:1], dtype=torch.float32, device=device)
    y_in = torch.tensor(surface_points_inlet[:, 1:2], dtype=torch.float32, device=device)
    z_in = torch.tensor(surface_points_inlet[:, 2:3], dtype=torch.float32, device=device)

    with torch.no_grad():
        u_in, v_in, w_in, p_in = model.forward(x_in, y_in, z_in)

   
    inlet_u_err = u_in.abs().mean().item()
    inlet_v_err = v_in.abs().mean().item()
    inlet_w_err = (w_in + V).abs().mean().item()
    print(f"[BC-Inlet] mean |u|={inlet_u_err:.3e}, |v|={inlet_v_err:.3e}, |w-V|={inlet_w_err:.3e}")

    # ---------- outlet: 压力 = 0 ----------
    x_out = torch.tensor(surface_points_outlet[:, 0:1], dtype=torch.float32, device=device)
    y_out = torch.tensor(surface_points_outlet[:, 1:2], dtype=torch.float32, device=device)
    z_out = torch.tensor(surface_points_outlet[:, 2:3], dtype=torch.float32, device=device)

    with torch.no_grad():
        u_out, v_out, w_out, p_out = model.forward(x_out, y_out, z_out)

    outlet_p_err = p_out.abs().mean().item()
    print(f"[BC-Outlet] mean |p|={outlet_p_err:.3e}")

check_bc_errors(pinn_model)


[BC-Wall] mean |u|=9.759e-03, |v|=1.439e-02, |w|=1.300e-02
[BC-Inlet] mean |u|=1.188e-02, |v|=1.234e-02, |w-V|=4.621e-02
[BC-Outlet] mean |p|=9.711e-04


In [197]:
def compute_div_on_volume(model, volume_points):
    device = next(model.net.parameters()).device

    if isinstance(volume_points, torch.Tensor):
        pts = volume_points.to(device).clone().detach().requires_grad_(True)
    else:
        pts = torch.tensor(volume_points, dtype=torch.float32, device=device, requires_grad=True)

    x = pts[:, 0:1]
    y = pts[:, 1:2]
    z = pts[:, 2:3]

    u, v, w, p = model.forward(x, y, z)

    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, torch.ones_like(v), create_graph=True)[0]
    w_z = torch.autograd.grad(w, z, torch.ones_like(w), create_graph=True)[0]

    div_res = u_x + v_y + w_z

    with torch.no_grad():
        div_np = div_res.detach().cpu().numpy().flatten()
        x_np = x.detach().cpu().numpy().flatten()
        y_np = y.detach().cpu().numpy().flatten()
        z_np = z.detach().cpu().numpy().flatten()

    return x_np, y_np, z_np, div_np
compute_div_on_volume(pinn_model, volume_points)

(array([-0.34503594,  0.18447907, -0.33622226, -0.4646458 , -0.6753007 ,
         0.59787095, -0.40263215, -0.8711678 , -0.4497568 , -0.37743792,
         0.08676347,  0.45963645, -0.65216935,  0.0347143 , -0.1773826 ,
        -0.02970922,  0.2808396 ,  0.27152574, -0.19172563, -0.44392732,
        -0.29528552, -0.87457436,  0.18560693,  0.09224759, -0.4313036 ,
         0.660567  , -0.41951084, -0.40400574,  0.09437928, -0.9561819 ,
        -0.05095849, -0.4316805 ,  0.67949075,  0.7926353 , -0.43865377,
        -0.9653714 ,  0.46509138,  0.17425677,  0.67149466, -0.44822192,
         0.3988929 ,  0.1056514 ,  0.04698248, -0.37947652,  0.43295747,
         0.74180883,  0.5333926 , -0.11332618,  0.87730825,  0.6280946 ,
         0.32579288,  0.55380344, -0.06224649, -0.7037104 ,  0.23588432,
         0.40639615,  0.4437458 ,  0.64895946,  0.54352635, -0.99195075,
         0.06153855,  0.95452684, -0.00619511, -0.51625526, -0.76523465,
         0.23647474,  0.5429259 , -0.9819912 ,  0.1